# Test SOFA Polars Implementation

This notebook tests the `compute_sofa_polars()` function using CLIFpy's built-in demo data.


In [ ]:
# Setup: Import required packages
import sys
from pathlib import Path

# Add the project root to path to ensure we're using the local clifpy
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import polars as pl
from datetime import datetime, timedelta
import pandas as pd

# Import clifpy modules
from clifpy import compute_sofa_polars
from clifpy.data.loader import _get_demo_data_path

print(f"Project root: {project_root}")
print(f"Using clifpy from: {project_root / 'clifpy'}")


## 1. Check Demo Data Availability

First, let's verify the demo data is available and see what we're working with.


In [ ]:
# Get demo data path
demo_data_path = _get_demo_data_path()
print(f"Demo data directory: {demo_data_path}")

# List available files
import os
demo_files = sorted(os.listdir(demo_data_path))
print(f"\nAvailable demo files ({len(demo_files)}):")
for f in demo_files:
    file_path = os.path.join(demo_data_path, f)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"  - {f} ({size_mb:.2f} MB)")


## 2. Explore Hospitalization Data

Let's look at the hospitalization data to understand the time ranges and create appropriate cohort windows.


In [ ]:
# Load hospitalization data to understand available time ranges
hosp_df = pl.read_parquet(os.path.join(demo_data_path, "clif_hospitalization.parquet"))
print(f"Hospitalization data shape: {hosp_df.shape}")
print(f"\nColumns: {hosp_df.columns}")
print(f"\nSample data:")
hosp_df.head(2)


In [ ]:
# Get unique hospitalization IDs
hospitalization_ids = hosp_df['hospitalization_id'].unique().to_list()
print(f"Number of unique hospitalizations: {len(hospitalization_ids)}")
print(f"Sample IDs: {hospitalization_ids[:5]}")


## 3. Create Cohort DataFrame

Create a cohort with time windows for SOFA computation. We'll use the admission times from hospitalization data and create 24-hour windows.

In [ ]:
# Create cohort DataFrame from hospitalization data
# Use admission_dttm as start and add 24 hours for end

# First, let's see what datetime columns are available
print("Datetime columns in hospitalization:")
for col in hosp_df.columns:
    if 'dttm' in col.lower():
        print(f"  - {col}: {hosp_df[col].dtype}")

In [ ]:
# Create cohort with 24-hour windows starting from admission
cohort_df = hosp_df.select([
    pl.col('hospitalization_id').cast(pl.Utf8),
    pl.col('admission_dttm').alias('start_dttm'),
    (pl.col('admission_dttm') + pl.duration(hours=24)).alias('end_dttm')
]).drop_nulls()

print(f"Cohort shape: {cohort_df.shape}")
print(f"\nCohort sample:")
cohort_df.head()


## 4. Compute SOFA Scores with Polars

Now let's run the `compute_sofa_polars()` function!


In [ ]:
%%time
# Enable logging to see what's happening
import logging
logging.basicConfig(level=logging.INFO)

# Compute SOFA scores
sofa_scores = compute_sofa_polars(
    data_directory=demo_data_path,
    cohort_df=cohort_df,
    filetype='parquet',
    id_name='hospitalization_id',
    extremal_type='worst',
    fill_na_scores_with_zero=True,
    remove_outliers=True,
    timezone='UTC'  # Demo data is in UTC
)

print(f"\n✅ SOFA computation complete!")
print(f"Result shape: {sofa_scores.shape}")


## 5. Examine Results

Let's look at the SOFA scores and their component values.


In [ ]:
# Show all columns in the result
print("Result columns:")
for i, col in enumerate(sofa_scores.columns):
    print(f"  {i+1}. {col}")


In [ ]:
# Display SOFA scores (main component scores and total)
sofa_cols = ['hospitalization_id', 'sofa_cv_97', 'sofa_coag', 'sofa_liver', 
             'sofa_resp', 'sofa_cns', 'sofa_renal', 'sofa_total']
available_cols = [col for col in sofa_cols if col in sofa_scores.columns]

print("SOFA Component Scores:")
sofa_scores.select(available_cols)


In [ ]:
# Summary statistics for SOFA total score
print("SOFA Total Score Distribution:")
print(sofa_scores['sofa_total'].describe())


In [ ]:
# Show intermediate clinical values used for SOFA calculation
intermediate_cols = ['hospitalization_id', 'map', 'platelet_count', 'bilirubin_total', 
                     'creatinine', 'gcs_total', 'p_f', 'fio2_set', 'device_category']
available_intermediate = [col for col in intermediate_cols if col in sofa_scores.columns]

print("Intermediate Clinical Values:")
sofa_scores.select(available_intermediate)


## 6. Validation

Let's validate that the SOFA scores are within expected ranges.


In [ ]:
# Validation tests
print("Running validation checks...\n")

# Check 1: All component scores should be 0-4
component_cols = ['sofa_cv_97', 'sofa_coag', 'sofa_liver', 'sofa_resp', 'sofa_cns', 'sofa_renal']
for col in component_cols:
    if col in sofa_scores.columns:
        min_val = sofa_scores[col].min()
        max_val = sofa_scores[col].max()
        valid = (min_val >= 0) and (max_val <= 4)
        status = "✅" if valid else "❌"
        print(f"{status} {col}: range [{min_val}, {max_val}] (expected 0-4)")

# Check 2: Total score should be 0-24
if 'sofa_total' in sofa_scores.columns:
    min_total = sofa_scores['sofa_total'].min()
    max_total = sofa_scores['sofa_total'].max()
    valid = (min_total >= 0) and (max_total <= 24)
    status = "✅" if valid else "❌"
    print(f"\n{status} sofa_total: range [{min_total}, {max_total}] (expected 0-24)")

# Check 3: No null values in SOFA scores (when fill_na_scores_with_zero=True)
null_counts = sofa_scores.select([pl.col(c).null_count().alias(c) for c in component_cols if c in sofa_scores.columns])
total_nulls = sum(null_counts.row(0))
status = "✅" if total_nulls == 0 else "❌"
print(f"\n{status} Null values in component scores: {total_nulls} (expected 0)")

print("\n" + "="*50)
print("🎉 SOFA Polars implementation test complete!")
